In [1]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName('LSP_PROJECT_2').getOrCreate()
import seaborn as sb
import matplotlib.pyplot as plt
from datetime import datetime

In [23]:
dataset_1=spark.read.csv("hdfs://localhost:9000/Dataset/covid_us_county.csv",inferSchema=True,header=True)

In [24]:
dataset_1.columns

['fips',
 'county',
 'state',
 'lat',
 'long',
 'date',
 'cases',
 'state_code',
 'deaths']

In [26]:
dataset_1.printSchema()

root
 |-- fips: string (nullable = true)
 |-- county: string (nullable = true)
 |-- state: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- date: string (nullable = true)
 |-- cases: integer (nullable = true)
 |-- state_code: string (nullable = true)
 |-- deaths: integer (nullable = true)



In [58]:
from pyspark.sql.functions import year
from pyspark.sql.functions import to_date
from pyspark.sql.functions import month
from pyspark.sql.functions import to_timestamp,date_format
from pyspark.sql.functions import col

In [68]:
df = dataset_1.withColumn('year',year(dataset_1.date))

In [69]:
df = df.withColumn('month',month(df.date))
df = df.withColumn("date",to_timestamp(col("date"))).withColumn("day", date_format(col("date"), "d"))
df.show()

+----+-------+-------+-----------+------------+-------------------+-----+----------+------+----+-----+---+
|fips| county|  state|        lat|        long|               date|cases|state_code|deaths|year|month|day|
+----+-------+-------+-----------+------------+-------------------+-----+----------+------+----+-----+---+
|1001|Autauga|Alabama|32.53952745|-86.64408227|2020-01-22 00:00:00|    0|        AL|     0|2020|    1| 22|
|1001|Autauga|Alabama|32.53952745|-86.64408227|2020-01-23 00:00:00|    0|        AL|     0|2020|    1| 23|
|1001|Autauga|Alabama|32.53952745|-86.64408227|2020-01-24 00:00:00|    0|        AL|     0|2020|    1| 24|
|1001|Autauga|Alabama|32.53952745|-86.64408227|2020-01-25 00:00:00|    0|        AL|     0|2020|    1| 25|
|1001|Autauga|Alabama|32.53952745|-86.64408227|2020-01-26 00:00:00|    0|        AL|     0|2020|    1| 26|
|1001|Autauga|Alabama|32.53952745|-86.64408227|2020-01-27 00:00:00|    0|        AL|     0|2020|    1| 27|
|1001|Autauga|Alabama|32.53952745|-86

In [70]:
df = df.drop('date')

In [71]:
df.show()

+----+-------+-------+-----------+------------+-----+----------+------+----+-----+---+
|fips| county|  state|        lat|        long|cases|state_code|deaths|year|month|day|
+----+-------+-------+-----------+------------+-----+----------+------+----+-----+---+
|1001|Autauga|Alabama|32.53952745|-86.64408227|    0|        AL|     0|2020|    1| 22|
|1001|Autauga|Alabama|32.53952745|-86.64408227|    0|        AL|     0|2020|    1| 23|
|1001|Autauga|Alabama|32.53952745|-86.64408227|    0|        AL|     0|2020|    1| 24|
|1001|Autauga|Alabama|32.53952745|-86.64408227|    0|        AL|     0|2020|    1| 25|
|1001|Autauga|Alabama|32.53952745|-86.64408227|    0|        AL|     0|2020|    1| 26|
|1001|Autauga|Alabama|32.53952745|-86.64408227|    0|        AL|     0|2020|    1| 27|
|1001|Autauga|Alabama|32.53952745|-86.64408227|    0|        AL|     0|2020|    1| 28|
|1001|Autauga|Alabama|32.53952745|-86.64408227|    0|        AL|     0|2020|    1| 29|
|1001|Autauga|Alabama|32.53952745|-86.64408

In [75]:
from pyspark.ml.feature import IndexToString, StringIndexer
from pyspark.ml.stat import Correlation
import pandas as pd
indexer = StringIndexer(inputCols=['lat',
  'long',
  'day',
  'month',
  'year',
  'cases',
  'state_code',
  'deaths'], outputCols=['latIndex',
  'longIndex',
  'dayIndex',
  'monthIndex',
  'yearIndex',
  'casesIndex',
  'state_codeIndex',
  'deathsIndex'])
indexed = indexer.fit(df).transform(df)
indexed.show()

+----+-------+-------+-----------+------------+-----+----------+------+----+-----+---+---------------+---------+---------+----------+--------+-----------+--------+----------+
|fips| county|  state|        lat|        long|cases|state_code|deaths|year|month|day|state_codeIndex|yearIndex|longIndex|monthIndex|latIndex|deathsIndex|dayIndex|casesIndex|
+----+-------+-------+-----------+------------+-----+----------+------+----+-----+---+---------------+---------+---------+----------+--------+-----------+--------+----------+
|1001|Autauga|Alabama|32.53952745|-86.64408227|    0|        AL|     0|2020|    1| 22|           20.0|      0.0|   1921.0|      10.0|   519.0|        0.0|    12.0|       0.0|
|1001|Autauga|Alabama|32.53952745|-86.64408227|    0|        AL|     0|2020|    1| 23|           20.0|      0.0|   1921.0|      10.0|   519.0|        0.0|    13.0|       0.0|
|1001|Autauga|Alabama|32.53952745|-86.64408227|    0|        AL|     0|2020|    1| 24|           20.0|      0.0|   1921.0|   

In [76]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler

In [78]:
#Divisons le   dataset , 80% pour les données d'entrainement et 20% pour Les données de test
splitedData = indexed.randomSplit([0.8, 0.2]) 
trainData = splitedData[0] #index 0 = data d'entrainement
testData = splitedData[1] #index 1 = data de test
train_rows = trainData.count()
test_rows = testData.count()
print ("Train data rows:", train_rows, "; Test data rows:", test_rows)

Train data rows: 972936 ; Test data rows: 242824


In [80]:
#Preparons le  Dataset
#Nous allons créer un vecteur  feature 
# Nous aurons deux grandes classes (died , discharged)
assembler = VectorAssembler(inputCols = [
  'latIndex',
  'longIndex',
  'dayIndex',
  'monthIndex',
  'yearIndex',
  'casesIndex',
  'state_codeIndex',
  'deathsIndex'], outputCol="features")
trainDataFinal = assembler.transform(
    trainData).select(col("features"), col("deathsIndex").alias("label"))
trainDataFinal.show(truncate=False, n=2)

+-----------------------------------------+-----+
|features                                 |label|
+-----------------------------------------+-----+
|[1834.0,847.0,13.0,10.0,0.0,0.0,50.0,0.0]|0.0  |
|[1834.0,847.0,15.0,10.0,0.0,0.0,50.0,0.0]|0.0  |
+-----------------------------------------+-----+
only showing top 2 rows



In [82]:
# Nous avons choisis la regression logistique qui est une technique de classification 
classifier = LogisticRegression(
labelCol="label",featuresCol="features",maxIter=10,regParam=0.2)
#Entrainement 
model = classifier.fit(trainDataFinal)
print ("Classifier model well is trained!")

###Données d'entrainement 
testDataFinal = assembler.transform(
    testData).select(col("features"), col("deathsIndex").alias("trueLabel"))
testDataFinal.show(20)

### Prédiction du model 
predict = model.transform(testDataFinal)
predictFinal = predict.select(
    "features", "prediction", "probability", "trueLabel")
predictFinal.show(truncate=False, n=2)
predict.show(truncate=False, n=2)

### Calcul des performances du model 
correctPredict = predictFinal.filter(
    predictFinal['prediction'] == predictFinal['trueLabel']).count()
totalData = predictFinal.count()
print("correct prediction:", correctPredict, ", total data:", totalData, 
      ", accuracy:", correctPredict/totalData)

Classifier model well is trained!
+--------------------+---------+
|            features|trueLabel|
+--------------------+---------+
|[1834.0,847.0,12....|      0.0|
|[1834.0,847.0,14....|      0.0|
|[1834.0,847.0,6.0...|      0.0|
|[1834.0,847.0,27....|      0.0|
|[1834.0,847.0,28....|      0.0|
|[1834.0,847.0,18....|      0.0|
|[1834.0,847.0,23....|      0.0|
|[1834.0,847.0,25....|      0.0|
|[1834.0,847.0,3.0...|      0.0|
|[1834.0,847.0,6.0...|      0.0|
|[1834.0,847.0,23....|      0.0|
|[1834.0,847.0,25....|      0.0|
|[1834.0,847.0,14....|      0.0|
|[1834.0,847.0,16....|      0.0|
|[1834.0,847.0,24....|      4.0|
|[1834.0,847.0,26....|      4.0|
|[1834.0,847.0,27....|     12.0|
|[1834.0,847.0,13....|     15.0|
|[1834.0,847.0,0.0...|     28.0|
|[1834.0,847.0,11....|     29.0|
+--------------------+---------+
only showing top 20 rows

+-----------------------------------------+----------+----------------------------------------------------------------------------------------------

+-----------------------------------------+---------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

correct prediction: 105461 , total data: 242824 , accuracy: 0.434310447072777
